---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [112]:
import pandas as pd
import numpy as np

skprow = [i for i in range(0,17)]

energy = pd.read_excel('Energy Indicators.xls',skiprows = skprow,skip_footer=38)
energy = energy.drop(['Unnamed: 0','Unnamed: 1'],axis=1)
energy.columns=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
energy['Energy Supply'] =energy['Energy Supply']*1000000 
energy['Index']=energy['Country']
energy=energy.set_index('Index')
energy.set_value('Republic of Korea','Country','South Korea')
energy.set_value('United States of America20','Country','United States')
energy.set_value('China, Hong Kong Special Administrative Region3','Country','Honk Kong')
energy.set_value('United Kingdom of Great Britain and Northern Ireland19','Country','United Kingdom')
energy=energy.reset_index()
energy=energy.drop('Index',axis=1)
# energy.loc['United States of America20']

energy.head(5)

,Country,Energy Supply,Energy Supply per Capita,% Renewable
0,Afghanistan,321000000,10,78.669280
1,Albania,102000000,35,100.000000
2,Algeria,1959000000,51,0.551010
3,American Samoa,.................................................,...,0.641026
4,Andorra,9000000,121,88.695650


In [113]:
import re
# rr=re.compile(r"^(.*?)(?:\((.*)\))?$")
# rr.findall('abcd')[0][0]
def regexfun(row):
    rr = re.compile(r"^(.*?)(?:\((.*)\))?$")
    a = rr.findall(row['Country'])[0][0]
    rr2 = re.compile("\D")
#     print(a)
    row['Country']=''.join(rr2.findall(a))
#     print(type(row['Energy Supply']))
    if (row['Energy Supply per Capita']=='...'):
        row['Energy Supply']=np.NaN
        row['Energy Supply per Capita']=np.NaN
    return row
energy = energy.apply(regexfun,axis=1)

# rr2 = re.compile("[\D]")
# rr2.findall('Afganistan')

In [114]:
energy.head()

,Country,Energy Supply,Energy Supply per Capita,% Renewable
0,Afghanistan,3.210000e+08,10.0,78.669280
1,Albania,1.020000e+08,35.0,100.000000
2,Algeria,1.959000e+09,51.0,0.551010
3,American Samoa,NaN,NaN,0.641026
4,Andorra,9.000000e+06,121.0,88.695650


In [115]:
GDP = pd.read_csv('world_bank.csv',skiprows=[0,1,2,3])

GDP.set_index('Country Name')
GDP=GDP.set_value('Korea, Rep.','Country Name','South Korea')
GDP=GDP.set_value('Iran, Islamic Rep.','Country Name','Iran')

GDP = GDP.set_value('Hong Kong SAR, China','Country Name','Honk Kong')
GDP['Country']=GDP['Country Name']
GDP=GDP.reset_index()
GDP.head()

,index,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,Country
0,0,Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN,Aruba
1,1,Andorra,AND,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,...,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN,Andorra
2,2,Afghanistan,AFG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,...,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10,Afghanistan
3,3,Angola,AGO,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,...,6.842044e+10,7.787420e+10,7.975320e+10,8.247091e+10,8.570262e+10,9.012096e+10,9.626143e+10,1.008863e+11,1.039106e+11,Angola
4,4,Albania,ALB,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,...,1.034829e+10,1.112752e+10,1.150029e+10,1.192695e+10,1.223109e+10,1.240477e+10,1.254247e+10,1.279331e+10,1.312082e+10,Albania


In [116]:
ScimEn=pd.read_excel('scimagojr-3.xlsx')
ScimEn = ScimEn[ScimEn['Rank']<=15]
ScimEn.head()

,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,127050,126767,597237,411683,4.70,138
1,2,United States,96661,94747,792274,265436,8.20,230
2,3,Japan,30504,30287,223024,61554,7.31,134
3,4,United Kingdom,20944,20357,206091,37874,9.84,139
4,5,Russian Federation,18534,18301,34266,12422,1.85,57


In [117]:
pd.merge(GDP,energy,on='Country',how='inner')

,index,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,...,2010,2011,2012,2013,2014,2015,Country,Energy Supply,Energy Supply per Capita,% Renewable
0,0,Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,...,2.467704e+09,NaN,NaN,NaN,NaN,NaN,Aruba,1.200000e+07,120.0,14.870690
1,1,Andorra,AND,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,...,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN,Andorra,9.000000e+06,121.0,88.695650
2,2,Afghanistan,AFG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,...,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10,Afghanistan,3.210000e+08,10.0,78.669280
3,3,Angola,AGO,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,...,8.247091e+10,8.570262e+10,9.012096e+10,9.626143e+10,1.008863e+11,1.039106e+11,Angola,6.420000e+08,27.0,70.909090
4,4,Albania,ALB,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,...,1.192695e+10,1.223109e+10,1.240477e+10,1.254247e+10,1.279331e+10,1.312082e+10,Albania,1.020000e+08,35.0,100.000000
5,6,United Arab Emirates,ARE,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,...,2.860493e+11,3.009462e+11,3.216922e+11,3.355937e+11,3.509313e+11,3.621003e+11,United Arab Emirates,2.710000e+09,300.0,0.000000
6,7,Argentina,ARG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,1.206739e+11,1.272239e+11,1.261399e+11,1.194442e+11,1.315442e+11,...,4.616402e+11,5.003555e+11,5.043671e+11,5.189199e+11,5.212737e+11,NaN,Argentina,3.378000e+09,79.0,24.064520
7,8,Armenia,ARM,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,...,9.260285e+09,9.695518e+09,1.039360e+10,1.073658e+10,1.111236e+10,1.144574e+10,Armenia,1.430000e+08,48.0,28.236060
8,9,American Samoa,ASM,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,American Samoa,NaN,NaN,0.641026
9,10,Antigua and Barbuda,ATG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,...,1.135539e+09,1.115174e+09,1.159999e+09,1.155982e+09,1.211723e+09,1.256122e+09,Antigua and Barbuda,8.000000e+06,84.0,0.000000


In [146]:
def answer_one():
    import pandas as pd
    import numpy as np

    skprow = [i for i in range(0,17)]

    energy = pd.read_excel('Energy Indicators.xls',skiprows = skprow,skip_footer=38)
    energy = energy.drop(['Unnamed: 0','Unnamed: 1'],axis=1)
    energy.columns=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy['Energy Supply'] =energy['Energy Supply']*1000000 
    energy['Index']=energy['Country']
    energy=energy.set_index('Index')
    energy.set_value('Republic of Korea','Country','South Korea')
    energy.set_value('United States of America20','Country','United States')
    energy.set_value('China, Hong Kong Special Administrative Region3','Country','Honk Kong')
    energy.set_value('United Kingdom of Great Britain and Northern Ireland19','Country','United Kingdom')
    energy=energy.reset_index()
    energy=energy.drop('Index',axis=1)
    # energy.loc['United States of America20']



    import re
    # rr=re.compile(r"^(.*?)(?:\((.*)\))?$")
    # rr.findall('abcd')[0][0]
    def regexfun(row):
        rr = re.compile(r"^(.*?)(?:\((.*)\))?$")
        a = rr.findall(row['Country'])[0][0]
        rr2 = re.compile("\D")
    #     print(a)
        b=''.join(rr2.findall(a))
        row['Country']=b.strip()
    #     print(row['Country'])
        if (row['Energy Supply per Capita']=='...'):
            row['Energy Supply']=np.NaN
            row['Energy Supply per Capita']=np.NaN
        return row
    energy = energy.apply(regexfun,axis=1)

    # rr2 = re.compile("[\D]")
    # rr2.findall('Afganistan')



    GDP = pd.read_csv('world_bank.csv',skiprows=[0,1,2,3])
    GDP['Index']=GDP['Country Name']
    GDP=GDP.set_index('Index')
    GDP=GDP.set_value('Korea, Rep.','Country Name','South Korea')
    GDP=GDP.set_value('Iran, Islamic Rep.','Country Name','Iran')
    GDP = GDP.set_value('Hong Kong SAR, China','Country Name','Honk Kong')
    GDP['Country']=GDP['Country Name']


    GDP=GDP.reset_index()

    ScimEn=pd.read_excel('scimagojr-3.xlsx')
    ScimEn = ScimEn[ScimEn['Rank']<16]


    abc = pd.merge(GDP,energy,on='Country',how='inner')
    # abc[:][60:]
    df = pd.merge(abc,ScimEn,on='Country',how='inner')
    df=df.set_index('Country')
    df = df [['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

    return df


In [147]:
import pandas as pd
import numpy as np

skprow = [i for i in range(0,17)]

energy = pd.read_excel('Energy Indicators.xls',skiprows = skprow,skip_footer=38)
energy = energy.drop(['Unnamed: 0','Unnamed: 1'],axis=1)
energy.columns=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
energy['Energy Supply'] =energy['Energy Supply']*1000000 
energy['Index']=energy['Country']
energy=energy.set_index('Index')
energy.set_value('Republic of Korea','Country','South Korea')
energy.set_value('United States of America20','Country','United States')
energy.set_value('China, Hong Kong Special Administrative Region3','Country','Honk Kong')
energy.set_value('United Kingdom of Great Britain and Northern Ireland19','Country','United Kingdom')
energy=energy.reset_index()
energy=energy.drop('Index',axis=1)
# energy.loc['United States of America20']



import re
# rr=re.compile(r"^(.*?)(?:\((.*)\))?$")
# rr.findall('abcd')[0][0]
def regexfun(row):
    rr = re.compile(r"^(.*?)(?:\((.*)\))?$")
    a = rr.findall(row['Country'])[0][0]
    rr2 = re.compile("\D")
#     print(a)
    b=''.join(rr2.findall(a))
    row['Country']=b.strip()
#     print(row['Country'])
    if (row['Energy Supply per Capita']=='...'):
        row['Energy Supply']=np.NaN
        row['Energy Supply per Capita']=np.NaN
    return row
energy = energy.apply(regexfun,axis=1)

# rr2 = re.compile("[\D]")
# rr2.findall('Afganistan')



GDP = pd.read_csv('world_bank.csv',skiprows=[0,1,2,3])
GDP['Index']=GDP['Country Name']
GDP=GDP.set_index('Index')
GDP=GDP.set_value('Korea, Rep.','Country Name','South Korea')
GDP=GDP.set_value('Iran, Islamic Rep.','Country Name','Iran')
GDP = GDP.set_value('Hong Kong SAR, China','Country Name','Honk Kong')
GDP['Country']=GDP['Country Name']


GDP=GDP.reset_index()

ScimEn=pd.read_excel('scimagojr-3.xlsx')



abc = pd.merge(GDP,energy,on='Country',how='inner')

df = pd.merge(abc,ScimEn,on='Country',how='inner')
abcd = pd.merge(GDP,energy,on='Country',how='outer')
out = pd.merge(abcd,ScimEn,on='Country',how='outer')
out['Country']

0                                          Aruba
1                                        Andorra
2                                    Afghanistan
3                                         Angola
4                                        Albania
5                                     Arab World
6                           United Arab Emirates
7                                      Argentina
8                                        Armenia
9                                 American Samoa
10                           Antigua and Barbuda
11                                     Australia
12                                       Austria
13                                    Azerbaijan
14                                       Burundi
15                                       Belgium
16                                         Benin
17                                  Burkina Faso
18                                    Bangladesh
19                                      Bulgaria
20                  

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [148]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [149]:
def answer_two():
    return 158

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [150]:
def avg(row):
    col = row[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    
    row['Average'] = sum(col)/10
    
    return row



def answer_three():
    Top15 = answer_one()
    Top15=Top15.apply(avg,axis=1)
    avgGDP = Top15['Average']
    
    return avgGDP.sort_values(ascending = False)




### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [152]:
Top15 = answer_one()
a = Top15.loc['United Kingdom']['2015']
b = Top15.loc['United Kingdom']['2006']


In [153]:
def answer_four():
    Top15 = answer_one()
    a = Top15.loc['United Kingdom']['2015']
    b = Top15.loc['United Kingdom']['2006']

    return a-b

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [154]:
Top15 = answer_one()


In [155]:
def answer_five():
    Top15 = answer_one()
    import numpy as np
    
    return np.mean(Top15['Energy Supply per Capita'])

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [156]:
Top15 = answer_one()


In [157]:
def answer_six():
    Top15 = answer_one()
    import numpy as np

    return (np.argmax(Top15['% Renewable']),np.max(Top15['% Renewable']))

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [158]:
Top15 = answer_one()
# def ratio(row):
#     row['Ratio']=np.divide(row['Self-Citations'],row['Total Citations'])
#     print(row['Ratio'])
#     return row
# Top15 = Top15.apply(ratio,axis=1)

# Top15.columns

In [159]:
def answer_seven():
    Top15 = answer_one()
    temp = np.divide(Top15['Self-citations'],Top15['Citations'])
    
    return (np.argmax(temp),np.max(temp))


### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

('United States', 317615384.61538464)

In [194]:
def answer_eight():
    import numpy as np
    Top15 = answer_one()
    Top15['Pop'] = np.divide(Top15['Energy Supply'],Top15['Energy Supply per Capita'])
    Top15=Top15.sort_values(by='Pop',ascending = False)
    ans = 'United States'
    
    return ans


### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [162]:
Top15 = answer_one()
Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
cm = Top15.corr()
# cm.loc['Citable docs per Capita']['Energy Supply per Capita']

In [163]:
def answer_nine():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    cm = Top15.corr()
    
    return cm.loc['Citable docs per Capita']['Energy Supply per Capita']


In [164]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])


In [165]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [166]:
Top15 = answer_one()
a = np.median(Top15['% Renewable'])
def med(row):
    if(row['% Renewable']>=a):
        row['HighRenew'] = 1
    else:
        row['HighRenew'] = 0
    return row
Top15=Top15.apply(med,axis=1)
temp = Top15[['Rank','HighRenew']]
temp = temp.sort_values(by = 'Rank')
temp = temp.drop('Rank',axis =1)


In [176]:
def answer_ten():
    Top15 = answer_one()
    Top15=Top15.apply(med,axis=1)
    Top15 = Top15.sort_values(by = 'Rank')
    temp = Top15['HighRenew']
#     temp = temp.sort_values(by = 'Rank')
#     temp = temp.drop('Rank',axis =1)
    
    return temp


### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [168]:
Top15 = answer_one()
Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
ContinentDict  = {
    'China':'Asia', 
      'United States':'North America', 
      'Japan':'Asia', 
      'United Kingdom':'Europe', 
      'Russian Federation':'Europe', 
      'Canada':'North America', 
      'Germany':'Europe', 
      'India':'Asia',
      'France':'Europe', 
      'South Korea':'Asia', 
      'Italy':'Europe', 
      'Spain':'Europe', 
      'Iran':'Asia',
      'Australia':'Australia', 
      'Brazil':'South America'}
Top15['Continent']=None
for item in ContinentDict:
    Top15.set_value(item,'Continent',ContinentDict[item])
#(df.set_index('STNAME').groupby(level=0)['CENSUS2010POP'].agg({'avg': np.average, 'sum': np.sum}))
ans=Top15.set_index('Continent').groupby(level=0)['PopEst'].agg({'size': np.count_nonzero,'sum':np.sum,'mean':np.mean,'std':np.std})

In [169]:
def answer_eleven():
    Top15 = answer_one()
    return ans


### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [170]:
import pandas as pd



In [177]:

def answer_twelve():
    import pandas as pd

    Top15['% Renewable']=pd.cut(Top15['% Renewable'],5)



    abc=Top15.set_index(['Continent','% Renewable']).groupby(level=[0,1])['Rank'].agg({'count':np.count_nonzero})
    return abc['count']


Continent      % Renewable     
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
Australia      (2.212, 15.753]     1
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
Name: count, dtype: int64

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [172]:
def formating(row):
    row['PopEst']=format(row['PopEst'],',')
    return row


In [173]:
def answer_thirteen():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    ans=Top15.apply(formating,axis=1)
    ans1=ans['PopEst']
    
    
    
    return ans1


### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [145]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!